## gpt-2 laguage modeling

In [23]:
from transformers import GPT2PreTrainedModel, AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")


In [24]:
input_ids = tokenizer.encode("Hello, my dog is cute", return_tensors="pt", add_special_tokens=True)
print(input_ids)
outputs = model(input_ids)
outputs.last_hidden_state.shape

tensor([[15496,    11,   616,  3290,   318, 13779]])


torch.Size([1, 6, 768])

In [26]:
tokenizer.vocab_size

50257

In [28]:
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py#L945
import torch
from torch import nn

lm_head = nn.Linear(768, tokenizer.vocab_size, bias=False)
hidden_states = outputs[0]

lm_logits = lm_head(hidden_states)
labels = input_ids

loss = None
if labels is not None:
	# Shift so that tokens < n predict n
	# сдвигаем токены на один влево, чтобы предсказывать следующий токен
	# пример
	# input_ids = [1, 2, 3, 4, 5]
	# outputs = model(input_ids)[0]
	# >>> outputs
	# [emb1, emb2, emb3, emb4, emb5] - эмбеддинги токенов
	# далее мы обрезаем токены
	# [emb1, emb2, emb3, emb4] -> [2, 3, 4, 5]
	# скажем emb1 это ембединг размерности словаря на основе которого мы бы хотели получить токен 2
	# Получается для языкового моделирования нам нужно преобразовать n-1, а последний токен нужен только лишь как id для вычисления лосса
	# поэтому наверное нужно изначально не тратить на него ресурсы?
	shift_logits = lm_logits[..., :-1, :].contiguous()
	shift_labels = labels[..., 1:].contiguous()
	# Flatten the tokens
	loss_fct = nn.CrossEntropyLoss()
	loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
	print(loss)


tensor(16.3760, grad_fn=<NllLossBackward0>)


In [30]:
lm_logits.shape

torch.Size([1, 6, 50257])

In [53]:
temp = torch.randn((1, 7, 3))
temp_logits = nn.Linear(3, 5)(temp)
temp_logits

tensor([[[-0.5206, -0.3519,  0.0869,  0.1716, -0.2535],
         [ 0.2199, -0.6840, -0.3358, -0.1752,  0.3698],
         [-0.5648, -0.7970,  0.1583,  0.7411, -0.3949],
         [-1.0351, -0.8233, -0.5335, -1.1948, -0.6986],
         [-0.7387,  1.3585,  0.1182, -1.4637, -0.0853],
         [-0.7226, -1.4413,  0.0575,  1.0104, -0.6493],
         [-0.2791, -0.3803, -0.4489, -1.0014,  0.0253]]],
       grad_fn=<ViewBackward0>)

In [54]:
temp_logits[..., :-1, :].contiguous()

tensor([[[-0.5206, -0.3519,  0.0869,  0.1716, -0.2535],
         [ 0.2199, -0.6840, -0.3358, -0.1752,  0.3698],
         [-0.5648, -0.7970,  0.1583,  0.7411, -0.3949],
         [-1.0351, -0.8233, -0.5335, -1.1948, -0.6986],
         [-0.7387,  1.3585,  0.1182, -1.4637, -0.0853],
         [-0.7226, -1.4413,  0.0575,  1.0104, -0.6493]]],
       grad_fn=<SliceBackward0>)

In [55]:
temp_logits[..., :-1, :].contiguous().shape

torch.Size([1, 6, 5])

In [43]:
labels

tensor([[15496,    11,   616,  3290,   318, 13779]])

In [44]:
labels[..., 1:]

tensor([[   11,   616,  3290,   318, 13779]])